In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import time
import ccxt


import sys
sys.path.append("..") 
from data_provider import read_data, load_data_from_exchange
from tp_utils import daily_returns,  create_market_buy_order,  create_market_sell_order, get_balance
from portfolio_tools import *
from tp_config import *


In [2]:
main_api = "K6oI8dnJvHAbw7UMhRp2dqJ29oLidkR56PZqMpw9yAvNqmu0ZorHlPI8L8KIDHBh"
main_secret = "lOrRq2zRybjZLXX8o7BsA0QOTHeGNLt87mPFje4gPHAPIqyaMCFWMVGNlJXHuikO"


pd.set_option('precision', 9)

#exchange = ccxt.binance({'apiKey': kshao_api, 'secret': kshao_secret})

exchange = ccxt.binance({'apiKey': main_api, 'secret': main_secret})


In [3]:
def sell_portfolio(exchange, portfolio, balance, debug = True):
    df_bal = get_balance(exchange)

    for market in portfolio.keys():
       # print(market)
        asset = market.split('-')[0]
        amount_in_wallet = float(df_bal[df_bal['asset'] == asset]['free'].values[0])
        amount_in_portfolio = portfolio[market]

        if amount_in_portfolio > amount_in_wallet:
            amount = amount_in_wallet
        else:
            amount = amount_in_portfolio

        if debug:
            ticker = market.replace('-','/')
            price_sell = exchange.fetch_ticker(ticker)['last']
        else:
            price_sell = create_market_sell_order(exchange, market, amount) 

        balance = balance + amount * price_sell
    return balance
     
def buy_portfolio(exchange, portfolio, balance, debug = True):
    for market in portfolio.keys():
        amount = portfolio[market]
#        print(market, amount)
        if debug:
            ticker = market.replace('-','/')
            price_buy = exchange.fetch_ticker(ticker)['last']
        else:
            price_buy =  create_market_buy_order(exchange, market, amount)
        balance = balance - price_buy*amount
    return balance   

In [4]:
pair = 'BTC'
tf = 60
timeframes = {'1h':tf}
balance = 0.1
lookback = 77
horizon = 7


index_assets = pd.read_csv('index_assets.csv')['asset'].tolist()

markets = [s + '-' + pair for s in index_assets]
start_timestamp = exchange.parse8601('2021-05-01 00:00:00')
data_path = DATA_PATH_CRYPTO + '/' + pair

portfolio = {}

In [5]:
sleep_time = 30
work = True
while work:
    time.sleep(sleep_time)
    current_time = datetime.now()
    if current_time.minute != 0:
        continue
    print("")        
    print("#----------------------------------------------------------------------#")        
    print("Current time:", current_time)    
    
    ##Sell portfolio
    balance = sell_portfolio(exchange, portfolio, balance, debug = False)

    print('Current balance:', balance)  
    
    ## Load data
    load_data_from_exchange(exchange, markets, timeframes, start_timestamp, data_path, verbose = False)
    df_prices =  load_data_for_portfolio(markets, '1h', verbose = False)   
    print(datetime.fromtimestamp(df_prices.index[-1]/1000))

    ## set_portfolio
    print('\n', "Set new portfolio")
    df_period = df_prices.iloc[-lookback:]
    ef = calc_frontier(df_period, 'ledoit_wolf')
    
    #Calc weights    
    try:
        dfw = calc_weights(ef, 'max_sharpe', 0) 
    except:
        pass
    print(dfw)
    portfolio = set_new_portfolio(exchange, dfw, balance)
    #print(portfolio)
    
    ## Buy portfolio
    balance = buy_portfolio(exchange, portfolio, balance, debug = False)
    print("Balance after buy:", balance)   


Current time: 2021-05-13 08:00:23.812957

#----------------------------------------------------------------------#
Current balance: 0.1
Done
Time taken = 0.010  hours
2021-05-13 08:00:00

 Set new portfolio
                W
IRIS-BTC  0.94675
NANO-BTC  0.29203
Balance after buy: -0.024146607431935255
Current time: 2021-05-13 09:00:07.603137
IRIS-BTC
NANO-BTC

#----------------------------------------------------------------------#
Current balance: 0.09539050505806475
Done
Time taken = 0.010  hours
2021-05-13 09:00:00

 Set new portfolio
                 W
ALGO-BTC   0.11666
1INCH-BTC  0.00246
BAT-BTC    0.03422
BNT-BTC    0.02162
BAL-BTC    0.01050
CRV-BTC    0.16420
DATA-BTC   0.01478
DASH-BTC   0.02389
DGB-BTC    0.00222
EGLD-BTC   0.05744
FIL-BTC    0.01902
GVT-BTC    0.05515
IOST-BTC   0.05800
IOTX-BTC   0.01194
LTC-BTC    0.04297
KAVA-BTC   0.01196
KNC-BTC    0.02606
MANA-BTC   0.03696
NANO-BTC   0.04654
PERL-BTC   0.01943
UTK-BTC    0.00182
VIDT-BTC   0.06151
SAND-BTC   0.02293
S

KeyboardInterrupt: 

In [ ]:
load_data_from_exchange(exchange, markets, timeframes, start_timestamp, data_path, verbose = True)

In [9]:
df_prices =  load_data_for_portfolio(markets, '1h', verbose = False)   
print(datetime.fromtimestamp(df_prices.index[-1]/1000))

2021-05-13 04:00:00


In [6]:
balance = sell_portfolio(exchange, portfolio, balance, debug = False)

IOST-BTC
IOTX-BTC
IOTA-BTC
STPT-BTC
XVS-BTC


### Start buy

In [10]:
## Load data
load_data_from_exchange(exchange, markets, timeframes, start_timestamp, data_path, verbose = False)
df_prices =  load_data_for_portfolio(markets, '1h', verbose = False)   
print(datetime.fromtimestamp(df_prices.index[-1]/1000))

## set_portfolio
print('\n', "Set new portfolio")
df_period = df_prices.iloc[-lookback:]
ef = calc_frontier(df_period, 'ledoit_wolf')

#Calc weights    
try:
    dfw = calc_weights(ef, 'max_sharpe', 0) 
except:
    pass
print(dfw)
portfolio = set_new_portfolio(exchange, dfw, balance)
#print(portfolio)

## Buy portfolio
balance = buy_portfolio(exchange, portfolio, balance, debug = False)
print("Balance after buy:", balance)   

Done
Time taken = 0.010  hours
2021-05-13 04:00:00

 Set new portfolio
                W
CRV-BTC   0.14341
EGLD-BTC  0.05216
IOTX-BTC  0.12128
KAVA-BTC  0.06527
KSM-BTC   0.34404
MKR-BTC   0.06717
NANO-BTC  0.06316
OMG-BTC   0.06585
STPT-BTC  0.07765
Balance after buy: -0.00022405152171896606
